<a href="https://www.kaggle.com/code/hrithikmajumdar/prostrate-cancer-grade-classification?scriptVersionId=187008658" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print("hello")

In [ ]:
train_df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')

In [1]:
pip install openslide-python

Note: you may need to restart the kernel to use updated packages.


**NORMAL Boudning Box way**

In [21]:
import os
import pandas as pd
import cv2
import numpy as np
import base64
import openslide

# Define paths to the folders
train_images_path = '/kaggle/input/prostate-cancer-grade-assessment/train_images'
train_label_masks_path = '/kaggle/input/prostate-cancer-grade-assessment/train_label_masks'

# Function to encode the image to base64
def encode_image_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    return image_base64

# Function to read images and masks using OpenSlide
def read_image_mask(image_id):
    image_path = os.path.join(train_images_path, f"{image_id}.tiff")
    mask_path = os.path.join(train_label_masks_path, f"{image_id}.tiff")
    
    # Debugging: Print file paths
    print(f"Reading image: {image_path}")
    print(f"Reading mask: {mask_path}")
    
    if not os.path.exists(image_path):
        print(f"Image not found for ID: {image_id}")
        return None, None
    if not os.path.exists(mask_path):
        print(f"Mask not found for ID: {image_id}")
        return None, None
    
    image_slide = openslide.OpenSlide(image_path)
    mask_slide = openslide.OpenSlide(mask_path)
    
    # Read the images at level 0 (full resolution)
    image = np.array(image_slide.read_region((0, 0), 0, image_slide.level_dimensions[0]))
    mask = np.array(mask_slide.read_region((0, 0), 0, mask_slide.level_dimensions[0]))
    
    # Convert mask image to grayscale
    mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2GRAY)
    
    return image, mask

# Function to process each image and mask
def process_image_mask(image_id):
    # Read the image and mask using OpenSlide
    image, mask = read_image_mask(image_id)
    
    if image is None or mask is None:
        return (image_id, None, None, None, None, None)
    
    # Debugging: Check if the images are read correctly
    print(f"Image shape for ID {image_id}: {image.shape}")
    print(f"Mask shape for ID {image_id}: {mask.shape}")
    
    # Find contours in the mask image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Debugging: Check if contours are found
    print(f"Number of contours found for ID {image_id}: {len(contours)}")
    
    # Assuming the largest contour is the one we want
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get the bounding box coordinates of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Crop the corresponding portion from the main image
        roi = image[y:y+h, x:x+w]
        
        # Debugging: Check the ROI shape
        print(f"ROI shape for ID {image_id}: {roi.shape}")
        
        # Encode the ROI to base64
        roi_base64 = encode_image_base64(roi)
        
        return (image_id, x, y, w, h, roi_base64)
    else:
        print(f"No contours found for ID {image_id}")
        return (image_id, None, None, None, None, None)

# List all files in the train_label_masks directory
mask_files = set(os.path.splitext(filename.lower())[0] for filename in os.listdir(train_label_masks_path))
image_files = set(os.path.splitext(filename.lower())[0] for filename in os.listdir(train_images_path))

# Get the list of common image IDs (assuming the filenames without extension are the image IDs)
image_ids = image_files.intersection(mask_files)

print(f"Total images found: {len(image_files)}")
print(f"Total masks found: {len(mask_files)}")
print(f"Total common IDs: {len(image_ids)}")

# Process each image and mask and collect the results
results = []
for image_id in image_ids:
    results.append(process_image_mask(image_id))

# Create a DataFrame to hold the results
roi_df = pd.DataFrame(results, columns=['image_id', 'x', 'y', 'width', 'height', 'roi_base64'])

# Save the DataFrame to a CSV file
roi_df.to_csv('rois_with_coordinates.csv', index=False)

# Debugging: Print the resulting DataFrame
print(roi_df.head())


Total images found: 10616
Total masks found: 10516
Total common IDs: 0
Empty DataFrame
Columns: [image_id, x, y, width, height, roi_base64]
Index: []


**BITWISE AND**

In [20]:
import os
import pandas as pd
import cv2
import numpy as np
import base64
import openslide

# Define paths to the folders
train_images_path = '/kaggle/input/prostate-cancer-grade-assessment/train_images'
train_label_masks_path = '/kaggle/input/prostate-cancer-grade-assessment/train_label_masks'

# Function to encode the image to base64
def encode_image_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    return image_base64

# Function to read images and masks using OpenSlide
def read_image_mask(image_id):
    image_path = os.path.join(train_images_path, f"{image_id}.tiff")
    mask_path = os.path.join(train_label_masks_path, f"{image_id}.tiff")
    
    if not os.path.exists(image_path):
        print(f"Image not found for ID: {image_id}")
        return None, None
    if not os.path.exists(mask_path):
        print(f"Mask not found for ID: {image_id}")
        return None, None
    
    image_slide = openslide.OpenSlide(image_path)
    mask_slide = openslide.OpenSlide(mask_path)
    
    # Read the images at level 0 (full resolution)
    image = np.array(image_slide.read_region((0, 0), 0, image_slide.level_dimensions[0]))
    mask = np.array(mask_slide.read_region((0, 0), 0, mask_slide.level_dimensions[0]))
    
    # Convert mask image to grayscale
    mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2GRAY)
    
    return image, mask

# Function to process each image and mask
def process_image_mask(image_id):
    # Read the image and mask using OpenSlide
    image, mask = read_image_mask(image_id)
    
    if image is None or mask is None:
        return (image_id, None, None, None, None, None)
    
    # Ensure mask is binary
    _, mask_binary = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
    
    # Perform bitwise AND operation
    roi = cv2.bitwise_and(image, image, mask=mask_binary)
    
    # Find contours in the mask image
    contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get the bounding box coordinates of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Crop the corresponding portion from the ROI
        cropped_roi = roi[y:y+h, x:x+w]
        
        # Encode the ROI to base64
        roi_base64 = encode_image_base64(cropped_roi)
        
        return (image_id, x, y, w, h, roi_base64)
    else:
        return (image_id, None, None, None, None, None)

# List all files in the train_label_masks directory
mask_files = set(os.path.splitext(filename.lower())[0] for filename in os.listdir(train_label_masks_path))
image_files = set(os.path.splitext(filename.lower())[0] for filename in os.listdir(train_images_path))

# Get the list of common image IDs (assuming the filenames without extension are the image IDs)
image_ids = image_files.intersection(mask_files)

print(f"Total images found: {len(image_files)}")
print(f"Total masks found: {len(mask_files)}")
print(f"Total common IDs: {len(image_ids)}")

# Process each image and mask and collect the results
results = []
for image_id in image_ids:
    results.append(process_image_mask(image_id))

# Create a DataFrame to hold the results
roi_df = pd.DataFrame(results, columns=['image_id', 'x', 'y', 'width', 'height', 'roi_base64'])

# Save the DataFrame to a CSV file
roi_df.to_csv('rois_with_coordinates.csv', index=False)

# Debugging: Print the resulting DataFrame
print(roi_df.head())


Total images found: 10616
Total masks found: 10516
Total common IDs: 0
Empty DataFrame
Columns: [image_id, x, y, width, height, roi_base64]
Index: []


In [13]:
pip install tifffile

Note: you may need to restart the kernel to use updated packages.


In [17]:
import cv2
import pandas as pd
from tifffile import imread

# Define paths to folders
train_images_path = "/kaggle/input/prostate-cancer-grade-assessment/train_images"
train_label_masks_path = "/kaggle/input/prostate-cancer-grade-assessment/train_label_masks"

# Define empty list to store ROI data (x1, y1, x2, y2)
roi_data = []

# Loop through each mask image
for mask_filename in os.listdir(train_label_masks_path):
  #mask = imread(os.path.join(train_label_masks_path, mask_filename))
  # Load the mask image (assuming grayscale)
  mask = cv2.imread(os.path.join(train_label_masks_path, mask_filename), cv2.IMREAD_GRAYSCALE)

  # Find contours in the mask
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Assuming one ROI per mask (modify if needed)
  if len(contours) > 0:
    # Get bounding box of the first contour
    x, y, w, h = cv2.boundingRect(contours[0])

    # Extract corresponding image filename (assuming same names)
    image_filename = mask_filename.replace("_mask.png", ".png")  # Modify extension if needed

    # Load the corresponding image from train_images
    image = cv2.imread(os.path.join(train_images_path, image_filename))

    # Calculate ROI coordinates for the main image
    roi_x1, roi_y1 = x, y
    roi_x2, roi_y2 = x + w, y + h

    # Append ROI data to the list
    roi_data.append([roi_x1, roi_y1, roi_x2, roi_y2, image_filename])

# Create pandas dataframe from the list
df = pd.DataFrame(roi_data, columns=["x1", "y1", "x2", "y2", "image_filename"])

# Save the dataframe as a CSV file
df.to_csv("roi_data.csv", index=False)

print("ROI data saved to roi_data.csv!")


[ WARN:0@3698.016] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/prostate-cancer-grade-assessment/train_images/c699f572c40dbee492ca9ac0c1a0d9f7_mask.tiff'): can't open/read file: check file path/integrity
[ WARN:0@3698.527] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/prostate-cancer-grade-assessment/train_images/78138d4fc0b8fceb04138cd44d4a4fdd_mask.tiff'): can't open/read file: check file path/integrity
[ WARN:0@3699.195] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/prostate-cancer-grade-assessment/train_images/8bf80024b1a7f12ce52136af81a930df_mask.tiff'): can't open/read file: check file path/integrity
[ WARN:0@3703.924] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/prostate-cancer-grade-assessment/train_images/b4f3528c1ab8e7ae2faac094cdbe06c3_mask.tiff'): can't open/read file: check file path/integrity
[ WARN:0@3705.084] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/prostate-cancer-grade-assessment/train_images/1abb

error: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:79: error: (-215:Assertion failed) pixels <= CV_IO_MAX_IMAGE_PIXELS in function 'validateInputImageSize'


In [7]:
data=pd.read_csv('/kaggle/working/rois_with_coordinates.csv')
data

,image_id,x,y,width,height,roi_base64


In [ ]:
train_df

In [ ]:
def convert_gleason_score(score):
    try:
        parts = score.split('+')
        return int(parts[0]) + int(parts[1])
    except (ValueError, AttributeError):
        return np.nan

train_df['gleason_score_numeric'] = train_df['gleason_score'].apply(convert_gleason_score)

In [ ]:
train_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='isup_grade', data=train_df)
plt.title('Distribution of ISUP Grade')

for p in plt.gca().patches:
    plt.gca().annotate(f"{p.get_height()}", (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

plt.subplot(1, 2, 2)
sns.countplot(x='gleason_score_numeric', data=train_df)
plt.title('Distribution of Gleason Score (Numeric)')

for p in plt.gca().patches:
    plt.gca().annotate(f"{p.get_height()}", (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()


In [ ]:
import cv2

In [ ]:
image_dir = '/kaggle/input/prostate-cancer-grade-assessment/train_images/'

In [ ]:
image_size = 128

In [ ]:
image_ids = train_df['image_id'].iloc[:9].tolist()

In [ ]:
import openslide

In [ ]:
image_id = train_df['image_id'].iloc[0]
full_image_path = image_dir + image_id + '.tiff'

try:

    example = openslide.OpenSlide(full_image_path)

    clipped_example = example.read_region((5000, 5000), 0, (image_size, image_size))

    plt.imshow(clipped_example)
    plt.title(f"Image ID: {image_id}")
    plt.axis('off')

    example.close()
except Exception as e:
    print(f"Error loading image {image_id}: {e}")

plt.show()

In [ ]:
train_df["image_path"] = [image_dir+image_id+".tiff" for image_id in train_df["image_id"]]

In [ ]:
train_df

In [ ]:
sns.countplot(data=train_df, x='isup_grade')
plt.title('Distribution of ISUP grades')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(train_df['isup_grade'].value_counts(), labels=train_df['isup_grade'].unique(), autopct='%1.1f%%', startangle=140)
plt.title('Distribution of ISUP grades')
plt.axis('equal')  
plt.show()

In [ ]:
sns.countplot(data=train_df, x='data_provider')
plt.title('Distribution of Data Providers')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(train_df['data_provider'].value_counts(), labels=train_df['data_provider'].unique(), autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Data Providers')
plt.axis('equal')  
plt.show()

In [ ]:
sns.countplot(data=train_df, x='gleason_score_numeric')
plt.title('Distribution of Gleason Scores')
plt.show()

In [ ]:
correlation_matrix = train_df[['isup_grade', 'gleason_score_numeric']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
training_item_count = int(len(train_df) * 0.8)
validation_df = train_df[training_item_count:]
train_df = train_df[:training_item_count]

In [ ]:
def get_single_sample(image_path, image_size=256, training=False, display=False):
    image = openslide.OpenSlide(image_path)
    mask_path = image_path.replace("train_images", "train_label_masks").replace(".tiff", "_mask.tiff")
    mask = openslide.OpenSlide(mask_path)
    
    stacked_image = []
    groundtruth_per_image = []
    
    maximum_iteration = 0
    selected_sample = False
    while not selected_sample:
        sampling_start_x = randint(image_size, image.dimensions[0] - image_size)
        sampling_start_y = randint(image_size, image.dimensions[1] - image_size)

        clipped_sample = image.read_region((sampling_start_x, sampling_start_y), 0, (256, 256))
        clipped_array = np.asarray(clipped_sample)
        
        if (not np.all(clipped_array == 255) and np.std(clipped_array) > 20) or maximum_iteration > 200:
            if display:
                plt.imshow(clipped_sample)
                plt.show()
                
            sampled_image = clipped_array[:, :, :3]
            
            if training:
                clipped_mask = mask.read_region((sampling_start_x, sampling_start_y), 0, (256, 256))
                groundtruth_per_image.append(np.mean(np.asarray(clipped_mask)[:, :, 0]))
            
            selected_sample = True
        maximum_iteration += 1
    
    if training: 
        return np.array(sampled_image), np.array(groundtruth_per_image)
    else:
        return np.array(sampled_image)

In [ ]:
def get_random_samples(image_path, image_size=256, display=False):
    image = openslide.OpenSlide(image_path)
    stacked_image = []
    
    selected_samples = 0
    maximum_iteration = 0
    while selected_samples < 3:
        sampling_start_x = randint(image_size, image.dimensions[0] - image_size)
        sampling_start_y = randint(image_size, image.dimensions[1] - image_size)

        clipped_sample = image.read_region((sampling_start_x, sampling_start_y), 0, (256, 256))
        clipped_array = np.asarray(clipped_sample)
        
        if (not np.all(clipped_array == 255) and np.std(clipped_array) > 20) or maximum_iteration > 200:
            if display:
                plt.imshow(clipped_sample)
                plt.show()

            stacked_image.append(clipped_array[:, :, :3])
            selected_samples += 1
        maximum_iteration += 1
    return np.array(stacked_image)

In [ ]:
def custom_single_image_generator(image_path_list, batch_size=16):
    while True:
        for start in range(0, len(image_path_list), batch_size):
            X_batch = []
            Y_batch = []
            end = min(start + batch_size, training_item_count)

            image_info_list = [get_single_sample(image_path, training=True) for image_path in image_path_list[start:end]]
            X_batch = np.array([image_info[0]/255. for image_info in image_info_list])
            Y_batch = np.array([image_info[1] for image_info in image_info_list])
            
            yield X_batch, Y_batch 

In [ ]:
from random import randint 

In [ ]:
random_samples = get_random_samples(train_df.iloc[0].image_path, display=True)
print("Random samples shape:", random_samples.shape)

In [ ]:
image_path = train_df.iloc[0].image_path  
sample_image, groundtruth = get_single_sample(image_path, training=True, display=True)

print("Sample Image Shape:", sample_image.shape)
print("Ground Truth:", groundtruth)

In [ ]:
import cv2
import PIL
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.models import Sequential, Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
import gc
import skimage.io
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.python.keras import backend as K
sess = K.get_session()

In [ ]:
img=openslide.OpenSlide('/kaggle/input/prostate-cancer-grade-assessment/train_images/2fd1c7dc4a0f3a546a59717d8e9d28c3.tiff')
display(img.get_thumbnail(size=(512,512)))

In [ ]:
img.dimensions

In [ ]:
train_df['isup_grade'].value_counts()

In [ ]:
labels=[]
data=[]
data_dir='/kaggle/input/panda-resized-train-data-512x512/train_images/train_images/'
for i in range(train_df.shape[0]):
    data.append(data_dir + train_df['image_id'].iloc[i]+'.png')
    labels.append(train_df['isup_grade'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['isup_grade']=labels

In [ ]:
df

In [ ]:
def display_images(image_paths, num_images=5):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        image_path = image_paths[i]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
        axes[i].imshow(image)
        axes[i].axis('off')
        axes[i].set_title(f'Image {i+1}')
    plt.show()

image_paths = df['images'].tolist()

display_images(image_paths, num_images=5)

In [ ]:
df.to_csv('/kaggle/working/prostrate.csv')

In [ ]:
from PIL import Image

In [ ]:
sample_size = 900

grouped = df.groupby('isup_grade')

df_new = pd.DataFrame(columns=df.columns)

for isup_grade, group in grouped:
    if len(group) >= sample_size:
        sampled_group = group.sample(n=sample_size, random_state=42)
    else:
        sampled_group = group.sample(n=len(group), random_state=42)
    df_new = pd.concat([df_new, sampled_group])

df_new.reset_index(drop=True, inplace=True)

print("Shape of df_new:", df_new.shape)

In [ ]:
df_new

In [ ]:
train_val_df, test_df = train_test_split(df_new, test_size=0.15, random_state=42, stratify=df_new['isup_grade'])

In [ ]:
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42, stratify=train_val_df['isup_grade'])

In [ ]:
print(f"Total samples: {len(df_new)}")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

In [ ]:
import time
import shutil
import pathlib
import itertools
from PIL import Image

import cv2
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

In [ ]:
train_df['isup_grade'] = train_df['isup_grade'].astype(str)
val_df['isup_grade'] = val_df['isup_grade'].astype(str)
test_df['isup_grade'] = test_df['isup_grade'].astype(str)

In [ ]:
batch_size = 32
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator(rescale=1./255,rotation_range=20, width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen = tr_gen.flow_from_dataframe(train_df, x_col= 'images', y_col= 'isup_grade', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe(val_df, x_col= 'images', y_col= 'isup_grade', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe(test_df, x_col= 'images', y_col= 'isup_grade', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [ ]:
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
from tensorflow.keras.applications import InceptionV3

In [ ]:
from keras.layers import GlobalAveragePooling2D

def inceptionv3_model(num_classes=None):
    model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(model.output)  
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(model.input, output)
    return model

inceptionv3_conv = inceptionv3_model(6)

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
class KappaScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_gen = self.validation_data
        val_true = []
        val_pred = []
        for i in range(len(val_gen)):
            x_val, y_val = val_gen[i]
            val_true.extend(tf.argmax(y_val, axis=1).numpy())
            val_pred.extend(tf.argmax(self.model.predict(x_val), axis=1).numpy())

        kappa = cohen_kappa_score(val_true, val_pred)
        print(f"\nEpoch {epoch + 1} - Cohen's Kappa: {kappa:.4f}")

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
inceptionv3_conv.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

opt = SGD(learning_rate=0.001)
inceptionv3_conv.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
nb_epochs = 75
batch_size=16
nb_train_steps = train_df.shape[0]//batch_size
nb_val_steps=val_df.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
history = inceptionv3_conv.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=valid_gen,
    validation_steps=nb_val_steps)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
test_gen.reset()
Y_pred = inceptionv3_conv.predict(test_gen, steps=len(test_gen), verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
y_true = test_gen.classes

In [ ]:
class_labels = list(test_gen.class_indices.keys())

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
conf_matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class_report = classification_report(y_true, y_pred, target_names=class_labels)
print('Classification Report')
print(class_report)

In [ ]:
# kappa_callback = KappaScoreCallback(validation_data=valid_gen)

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
def vgg16_model(num_classes=None):
    model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = Flatten()(model.output)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(model.input, output)
    return model

vgg_conv = vgg16_model(6)

In [ ]:
# from sklearn.metrics import cohen_kappa_score

In [ ]:
# from tensorflow.keras.callbacks import Callback

class KappaScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_gen = self.validation_data
        val_true = []
        val_pred = []
        for i in range(len(val_gen)):
            x_val, y_val = val_gen[i]
            val_true.extend(tf.argmax(y_val, axis=1).numpy())
            val_pred.extend(tf.argmax(self.model.predict(x_val), axis=1).numpy())

        kappa = cohen_kappa_score(val_true, val_pred)
        print(f"\nEpoch {epoch + 1} - Cohen's Kappa: {kappa:.4f}")

In [ ]:
opt = Adam(learning_rate=0.001)
#opt = SGD(learning_rate=0.001)
vgg_conv.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

nb_epochs = 50
batch_size=16
nb_train_steps = train_df.shape[0]//batch_size
nb_val_steps=val_df.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
# kappa_callback = KappaScoreCallback(validation_data=valid_gen)

In [ ]:
history = vgg_conv.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=valid_gen,
    validation_steps=nb_val_steps)

history = inceptionv3_conv.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=valid_gen,
    validation_steps=nb_val_steps)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix

test_gen.reset()
Y_pred = inceptionv3_conv.predict(test_gen, steps=len(test_gen), verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
test_gen.reset()
Y_pred = vgg_conv.predict(test_gen, steps=len(test_gen), verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# y_true = test_gen.classes

In [ ]:
# class_labels = list(test_gen.class_indices.keys())

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
conf_matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class_report = classification_report(y_true, y_pred, target_names=class_labels)
print('Classification Report')
print(class_report)

In [ ]:
from tensorflow.keras.applications import MobileNet

def mobilenet_model(num_classes=None):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(base_model.input, output)
    return model

 = mobilenet_model(6)

In [ ]:
opt = Adam(learning_rate=0.001)
mobilenet_model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
test_gen.reset()
Y_pred = mobilenet_model.predict(test_gen, steps=len(test_gen), verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
history = mobilenet_model.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=valid_gen,
    validation_steps=nb_val_steps)

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
conf_matrix 

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class_report = classification_report(y_true, y_pred, target_names=class_labels)
print('Classification Report')
print(class_report)